### Installing Packages

In [2]:
!pip install PyMySQL

### Importing Packages

In [109]:
import pymysql
import pandas as pd
import datetime
import numpy as np

### Extracting data from AWS RDS

#### Creating a connection object

In [7]:
connection = pymysql.connect(host = 'orders-db-jsk.ccubfdmlmxi1.ap-south-1.rds.amazonaws.com',user = 'jskorders',password = '12345678')

#### Creating a cursor object

In [ ]:
cursor = connection.cursor()

#### getting data from customers, orders, returns and transactions tables

#### returns

In [38]:
sql = """SELECT * FROM etl.returns"""
cursor.execute(sql)

296

In [39]:
returns = pd.DataFrame(list(cursor.fetchall()))

In [55]:
returns.rename(columns = {0:'order_id',1:'return_reason'},inplace = True)

-------

#### orders

In [42]:
sql = """SELECT * FROM etl.orders"""
cursor.execute(sql)

5006

In [43]:
orders = pd.DataFrame(list(cursor.fetchall()))

In [56]:
orders.rename(columns = {0:'order_id',
                        1:'customer_id',
                        2:'order_status',
                        3:'order_purchase_date',
                        4:'order_approved_at',
                        5:'order_delivered_carrier_date',
                        6:'order_delivered_customer_date',
                        7:'order_estimated_delivery_date'},inplace = True)

----

#### transactions

In [46]:
sql = '''SELECT * FROM etl.transactions'''
cursor.execute(sql)

5910

In [47]:
transactions = pd.DataFrame(list(cursor.fetchall()))

In [57]:
transactions.rename(columns = {0:'order_id',
                              1:'ship_date',
                              2:'ship_mode',
                              3:'product_id',
                              4:'category',
                              5:'sub_category',
                              6:'product_name',
                              7:'sales',
                              8:'quantity',
                              9:'discount',
                              10:'profit'},inplace = True)

-----

#### Customers

In [50]:
sql = '''SELECT * FROM etl.customers'''
cursor.execute(sql)

332

In [51]:
customers = pd.DataFrame(list(cursor.fetchall()))

In [58]:
customers.rename(columns = {0:'customer_id',
                           1:'customer_email',
                           2:'customer_name',
                           3:'segment',
                           4:'country',
                           5:'city',
                           6:'state',
                           7:'postal_code',
                           8:'region'}, inplace = True)

-----

### Review the created dataframes from AWS RDS

In [60]:
customers.head(1)

,customer_id,customer_email,customer_name,segment,country,city,state,postal_code,region
0,AA-10375,stern@verizon.net,Allen Armold,Consumer,United States,Mesa,Arizona,85204,West


In [61]:
orders.head(1)

,order_id,customer_id,order_status,order_purchase_date,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,CA-2017-147039,AA-10315,delivered,2018-06-19,2018-06-19,2018-06-20,2018-06-27,2018-07-25


In [62]:
returns.head(1)

,order_id,return_reason
0,CA-2014-100762,Wrong Delivery


In [63]:
transactions.head(1)

,order_id,ship_date,ship_mode,product_id,category,sub_category,product_name,sales,quantity,discount,profit
0,CA-2014-100090,12-07-14,Standard Class,OFF-SU-10000646,Office Supplies,Supplies,Premier Automatic Letter Opener,502.488,3,0.2,-87.9354


-----

### Creating Datawarehouse tables

In [133]:
orders.order_approved_at.isnull().value_counts()

False    4996
True       10
Name: order_approved_at, dtype: int64

In [88]:
orders.columns

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_date',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date'],
      dtype='object')

### Replacing empty string to nan

In [122]:
orders.replace('',np.nan,inplace = True)

### Converting date strings into datetime objects

In [148]:
def date_convert(row):
    #function to convert datestring into datetime object.
    def convert(x):
        row[x] = datetime.datetime.strptime(row[x],'%Y-%m-%d')
    #list of columns containing date strings.
    date_col = ['order_purchase_date','order_approved_at', 'order_delivered_carrier_date',
                'order_delivered_customer_date', 'order_estimated_delivery_date']
    #For every row, we are iterating through     
    for item in date_col:
        if type(row[item]) == str:
            convert(item)
    #return the transformed row.    
    return row
    
orders.apply(date_convert,axis = 'columns')

,order_id,customer_id,order_status,order_purchase_date,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,CA-2017-147039,AA-10315,delivered,2018-06-19,2018-06-19,2018-06-20,2018-06-27,2018-07-25
1,CA-2015-121391,AA-10315,delivered,2017-11-24,2017-11-25,2017-11-30,2017-12-05,2017-12-08
2,CA-2014-128055,AA-10315,delivered,2018-05-14,2018-05-14,2018-05-14,2018-05-23,2018-06-05
3,CA-2016-103982,AA-10315,delivered,2018-04-09,2018-04-10,2018-04-10,2018-04-12,2018-04-19
4,CA-2014-138100,AA-10315,delivered,2017-12-04,2017-12-06,2017-12-08,2017-12-18,2018-02-13
...,...,...,...,...,...,...,...,...
5001,CA-2014-140487,NaN,delivered,2018-03-20,2018-03-20,2018-03-22,2018-04-17,2018-04-20
5002,CA-2014-131247,NaN,delivered,2018-04-05,2018-04-06,2018-04-10,2018-04-18,2018-04-26
5003,US-2016-106600,NaN,delivered,2018-03-08,2018-03-08,2018-03-09,2018-05-03,2018-03-29
5004,CA-2017-141201,NaN,delivered,2018-07-19,2018-07-21,2018-07-25,2018-07-30,2018-08-08


#### Inserting 4 new columns into orders dataframe

In [200]:
def new_col(row):
    
    if type(row['order_purchase_date']) == type(row['order_approved_at']) == datetime.datetime:
        row['order_to_approval_duration'] = (row['order_approved_at'] - row['order_purchase_date']).days
        return row
    else:
        row['order_to_approval_duration'] = np.nan
        return row
    
orders = orders.apply(new_col, axis = 'columns')

In [226]:
def new_col_1(row):
    
    if (type(row['order_approved_at']) != np.nan and type(row['order_delivered_carrier_date']) != np.nan):
        row['approval_to_carrier_duration'] = (row['order_delivered_carrier_date'] - row['order_approved_at']).days
        return row
    else:
        row['approval_to_carrier_duration'] = np.nan
        return row
    
orders = orders.apply(new_col_1, axis = 'columns')

In [229]:
def new_col_2(row):
    
    if (type(row['order_delivered_carrier_date']) != np.nan and type(row['order_delivered_customer_date']) != np.nan):
        row['carrier_to_customer_duration'] = (row['order_delivered_customer_date'] - row['order_delivered_carrier_date']).days
        return row
    else:
        row['carrier_to_customer_duration'] = np.nan
        return row
    
orders = orders.apply(new_col_2, axis = 'columns')

In [232]:
def new_col_3(row):
    
    if (type(row['order_delivered_customer_date']) != np.nan and type(row['order_estimated_delivery_date'] != np.nan)):
        row['delivery_delay'] = (row['order_estimated_delivery_date'] - row['order_delivered_customer_date']).days
        return row
    else:
        row['delivery_delay'] = np.nan
        return row
    
orders = orders.apply(new_col_3, axis = 'columns')

#### Order value and Order quantity

In [239]:
df_val_quant = transactions[['order_id','sales','quantity']].rename(columns = {'sales':'order_value','quantity':'order_quantity'})

In [240]:
df_val_quant.head()

,order_id,order_value,order_quantity
0,CA-2014-100090,502.488,3
1,CA-2014-100090,196.704,6
2,CA-2014-100328,3.928,1
3,CA-2014-100678,2.688,2
4,CA-2014-100678,317.058,3


In [255]:
order_with_value = pd.merge(orders,df_val_quant,how = 'inner',on = 'order_id')

In [262]:
order_with_return = pd.merge(order_with_value,returns,how='left',on='order_id')

In [274]:
def returned(row):
    if type(row['return_reason']) == float:
        row['returned'] = 'N'
        return row
    else:
        row['returned'] = 'Y'
        return row

order_with_return = order_with_return.apply(returned, axis = 'columns')

In [280]:
Order_Fact = order_with_return[['order_id','customer_id','order_to_approval_duration', 'approval_to_carrier_duration',
       'carrier_to_customer_duration', 'delivery_delay', 'order_value',
       'order_quantity','returned','return_reason']]

In [289]:
Order_Fact.to_csv('e:\\Users\\A.JOTHINAGARAJANA\\Downloads\\data\\Order_Fact.csv',index = False)

In [291]:
customers.to_csv('e:\\Users\\A.JOTHINAGARAJANA\\Downloads\\data\\Customer_Dim.csv',index = False)